### For Google Collab. You can set your own paths.

In [1]:
from google.colab import drive
import os

drive.mount("/content/drive") # Don't change this.

utils_path = "/content/drive" + "/My Drive/" + "TFM/python_visualization/"
os.chdir(utils_path)

from aux_funcs_vis import *

dataset_path = "/content/drive" + "/My Drive/" + "TFM/python_visualization/Thesis_defense_test_dataset"
os.chdir(dataset_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output

### For Notebooks locally. You can set your own paths.

In [ ]:
os.chdir('C:/Users/34692/Desktop/mgc/uc3m/2/TFM/TFM-foundation-model-wireless-signals/datasets/Thesis_defense_test_dataset')

print(f'Current directory:{os.getcwd()}')

## Run the following cell to find out what modulations are available

In [3]:
# In case current folder is not the datasets folder, add the path to the function: print_mods(path)
print_mods(os.getcwd())

Bluetooth_1 :
  Modulation Bluetooth BR 

OFDM_2 :
  Modulation OFDM 

PSK_1 :
  Modulation PSK 

QAM_1 :
  Modulation QAM 

WifiVHT_1 :
  Modulation WiFi VHT 



### Run the following cell and choose one of the modulations to find out more information about it

In [4]:
mods = get_mods(os.getcwd())
mods.insert(0, '---')
dropdown_mod = widgets.Dropdown(
    options=mods,
    description='Modulation:',
    default='---',
    disabled=False
)

output = widgets.Output()

def update_output(change):
    with output:
        clear_output(wait=True)
        print_mods_full(change['new'])

dropdown_mod.observe(update_output, names='value')

display(dropdown_mod, output)


Dropdown(description='Modulation:', options=('---', 'Bluetooth_1', 'OFDM_2', 'PSK_1', 'QAM_1', 'WifiVHT_1'), v…

Output()

### Now, let's visualize the signals. You can select between visualizing a single signal from one of the previous modulations or put together 2 signals and visualize the interference (along with the corresponding SoI)

After each run of the cell (and visualization) it is recommended to run the cell again

In [5]:
# Selector for visualization type
signal_type_selector = widgets.Select(
    options=['---', 'Single Signal', 'Interference'],
    description='Choose type:',
    value='---',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style={'description_width': '150px'}
)

# List of modulations with "---" at the beginning
mods = get_mods(os.getcwd())
mods.insert(0, '---')

# Dropdowns to select signals
dropdown_mod1 = widgets.Dropdown(
    options=mods,
    description='Signal of interest:',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style={'description_width': '150px'}
)

dropdown_mod2 = widgets.Dropdown(
    options=mods,
    description='Interfering signal:',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style={'description_width': '150px'}
)

# Slider to control the number of signals to visualize
numSignals = widgets.IntSlider(
    value=1, min=1, max=10, step=1,
    description='Num Signals:',
    disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='d',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '150px'}
)

# Slider to control the interference mix factor
mix_factor = widgets.FloatSlider(
    value=0.5, min=0, max=1.0, step=0.05,
    description='Mix Factor:',
    disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.2f',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '150px'}
)

# Button to execute visualization
run_button = widgets.Button(
    description="Visualize",
    button_style="success",
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'}
)

# Checkboxes to select visualization options
plot_time_checkbox = widgets.Checkbox(value=True, description="Visualize Time Domain waveform", layout=widgets.Layout(width='auto'), style={'description_width': '150px'})
plot_spectrogram_checkbox = widgets.Checkbox(value=True, description="Visualize Spectrogram", layout=widgets.Layout(width='auto'), style={'description_width': '150px'})
plot_constellation_checkbox = widgets.Checkbox(value=False, description="Visualize Constellation (Only for OFDM, Wifi NonHT or Bluetooth modulations)", layout=widgets.Layout(width='auto'), style={'description_width': '150px'})

# Output widget to display plots and errors
output = widgets.Output()


def execute_visualization(b):
    with output:
        clear_output(wait=True)

        # Ensure at least one visualization option is selected
        if not (plot_time_checkbox.value or plot_spectrogram_checkbox.value or plot_constellation_checkbox.value):
            print("ERROR: You must select at least one visualization option.")
            return

        if dropdown_mod1.value == '---':
            print("ERROR: You must select at least one modulation option.")
            return

        if (signal_type_selector.value == "Interference") and (dropdown_mod2.value == '---'):
            print("ERROR: You must select at least one modulation option for the interference. Otherwise, you can change to 'Single Signal'")
            return

        print("It may take a while for the visualizations to load...")

        batch_size = numSignals.value

        plot_time = plot_time_checkbox.value
        plot_spec = plot_spectrogram_checkbox.value
        plot_constel = plot_constellation_checkbox.value

        if signal_type_selector.value == "Single Signal":
            mod = dropdown_mod1.value
            h5file = mod + '.h5'
            jsonfile = mod + '.json'
            visualize_signal(h5file, jsonfile, batch_size,
                             plot_time=plot_time,
                             plot_spec=plot_spec,
                             plot_constel=plot_constel)

        elif signal_type_selector.value == "Interference":
            mod1 = dropdown_mod1.value
            mod2 = dropdown_mod2.value
            h5file1 = mod1 + '.h5'
            jsonfile1 = mod1 + '.json'
            h5file2 = mod2 + '.h5'
            jsonfile2 = mod2 + '.json'
            visualize_signal_mixed(h5file1, jsonfile1, h5file2, jsonfile2, batch_size,
                                   plot_time=plot_time,
                                   plot_spec=plot_spec,
                                   plot_constel=plot_constel,
                                   mix_factor=mix_factor.value)



run_button.on_click(execute_visualization)

def update_ui(change):
    with output:
        clear_output(wait=True)  # Clear everything before redisplaying the interface

    # Reset widget values
    dropdown_mod1.value = '---'
    dropdown_mod2.value = '---'
    numSignals.value = 1
    mix_factor.value = 0.5
    plot_time_checkbox.value = True
    plot_spectrogram_checkbox.value = True
    plot_constellation_checkbox.value = False

    # Display the main selector again
    clear_output(wait=True)
    display(signal_type_selector)

    # UI for a single signal
    if signal_type_selector.value == "Single Signal":
        display(dropdown_mod1, numSignals, plot_time_checkbox, plot_spectrogram_checkbox, plot_constellation_checkbox, run_button, output)

    # UI for interference of two signals
    elif signal_type_selector.value == "Interference":
        display(dropdown_mod1, dropdown_mod2, mix_factor, numSignals, plot_time_checkbox, plot_spectrogram_checkbox, plot_constellation_checkbox, run_button, output)

# Connect observer to reset the interface when the selector changes
signal_type_selector.observe(update_ui, names='value')

# Display the selector for the first time
display(signal_type_selector)
update_ui(None)

Select(description='Choose type:', index=2, layout=Layout(width='500px'), options=('---', 'Single Signal', 'In…

Dropdown(description='Signal of interest:', layout=Layout(width='500px'), options=('---', 'Bluetooth_1', 'OFDM…

Dropdown(description='Interfering signal:', layout=Layout(width='500px'), options=('---', 'Bluetooth_1', 'OFDM…

FloatSlider(value=0.5, continuous_update=False, description='Mix Factor:', layout=Layout(width='500px'), max=1…

IntSlider(value=1, continuous_update=False, description='Num Signals:', layout=Layout(width='500px'), max=10, …

Checkbox(value=True, description='Visualize Time Domain waveform', layout=Layout(width='auto'), style=Descript…

Checkbox(value=True, description='Visualize Spectrogram', layout=Layout(width='auto'), style=DescriptionStyle(…

Checkbox(value=False, description='Visualize Constellation (Only for OFDM, Wifi NonHT or Bluetooth modulations…

Button(button_style='success', description='Visualize', layout=Layout(width='600px'), style=ButtonStyle())

Output()